## The task in this notebook is to recreate the dataset used in the paper, by Kačar et al., 2023: Aircraft Accident Prediction Using Machine Learning Classification Algorithms.

### Data preprocessing procedures:	

0. merge all tables into one spreadsheet;
1. create the Severtiy Classes:
1. remove ‘homebuilt’ == Yes and ‘acft_category’ != ‘Airplane’ 
2. remove ‘Severity Class’ == NaN
3. remove column, if variance(column) == 0  (i.e. there is only a value in the column)
4. check column, if variance(column) ~ 0 (near-zero): consider transforming those columns to Categorical;
5. remove duplicated rows of data;
6. change wrong ‘flags’ to NaNs: 999 for gust_kts, -1 for apr_dist, outside of [-180,180] for longitude
7. IQR to remove data >99% (outliers);

8. IMPORTANT: Split Now, to avoid data Leakage. Keep 10/20% in a Holdout Set for final evaluation.

9.  Cross-Correlated features:
    1. Numerical: Pearson correlation and Spearman correlation. Remove: wx_dev_pt, as it is correlated with wx_temp
    2. Categorical: Phi-k correlation and Mutual Information. Nothing removed.
10. Relationship between features and labels (?):
    1. Numerical: ANOVA. Remove apt_dist, gust_kts, ev_time, ev_year, wind_dir_deg
    2. Categorical: Mutual Information. Remove ev_dow, ev_month.
11. Replace NaNs with median (numerical cols) and mode (categorical cols).
12. MinMaxScaler Normalise;
13. One-hot Encode Categorical features.

In [1]:
import numpy as np
import pandas as pd

### Opening every sheet of the NTSB dataset, using pandas:

This is not a common way to open datasets using pandas, but we need to do this for Excel files with multiple sheets.

It is more common to do:

``` dataframe = pd.read_excel(path_to_data)```, but this only reads the first sheet of the Excel file, if nothing else is specified.

In the case of **.csv** files, which are the most common, we can use:

```dataframe = pd.read_csv(path_to_data)```


In [2]:
# relative path from the folder containing this notebook:
path_to_data = '../data/NTSB avall file dataset.xlsx'

# initialise the Excel file:
ntsb = pd.ExcelFile(path_to_data)

# read every sheet in the Excel file:
ntsb1 = pd.read_excel(ntsb, 'Aircraft related')
ntsb2 = pd.read_excel(ntsb, 'Event related')
ntsb3 = pd.read_excel(ntsb, 'Event sequence')
ntsb4 = pd.read_excel(ntsb, 'Findings')
ntsb5 = pd.read_excel(ntsb, 'Engines')

In [4]:
print(len(ntsb1))
print(len(ntsb2))
print(len(ntsb3))
print(len(ntsb4))
print(len(ntsb5))

27844
27410
59327
65093
25266


In [40]:
common_columns = list(set(ntsb1.columns) & set(ntsb2.columns))

merged = pd.merge(ntsb1, ntsb2, how='inner', on=common_columns)
# merged = pd.merge(ntsb1, ntsb2, how='left', on=common_columns)
# merged = pd.merge(ntsb1, ntsb2, how='outer', on=common_columns)

In [5]:
merged = ntsb1.copy()

common_columns = list(set(merged.columns) & set(ntsb2.columns))
merged = pd.merge(merged, ntsb2, how='inner', on=common_columns)

common_columns = list(set(merged.columns) & set(ntsb3.columns))
merged = pd.merge(merged, ntsb3, how='inner', on=common_columns)

common_columns = list(set(merged.columns) & set(ntsb4.columns))
merged = pd.merge(merged, ntsb4, how='inner', on=common_columns)

common_columns = list(set(merged.columns) & set(ntsb5.columns))
merged = pd.merge(merged, ntsb5, how='inner', on=common_columns)

# for df in [ntsb2, ntsb3, ntsb4, ntsb5]:
#     common_columns = list(set(merged.columns) & set(df.columns))
#     print(common_columns)
#     merged = pd.merge(merged, df, how='inner', on=common_columns)

ValueError: You are trying to merge on object and float64 columns. If you wish to proceed you should use pd.concat